In [8]:
import topmost
from topmost.data import download_dataset
from topmost.preprocessing import Preprocessing
import os
import json
import pickle
device = "cuda" # or "cpu"
dataset_dir = "/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/models_preproc/iter_0/topmost"

In [2]:
import os
import re
import string
import gensim.downloader
from collections import Counter
import numpy as np
import scipy.sparse
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
dataset_dir = '/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/models_preproc/iter_0/topmost'

In [ ]:
# preprocess raw data
preprocessing = Preprocessing(vocab_size=10000)

rst = preprocessing.preprocess_jsonlist(dataset_dir=dataset_dir,label_name="label")

preprocessing.save(dataset_dir, **rst)

In [5]:
dataset = topmost.data.BasicDataset(dataset_dir, read_labels=True, device=device)

train_size:  52460
test_size:  13116
vocab_size:  10000
average length: 74.651


In [ ]:
# create a model
# model = topmost.models.SawETM(vocab_size=dataset.vocab_size, num_topics_list=[10, 50, 200], device=device)
model = topmost.models.HyperMiner(vocab_size=dataset.vocab_size, num_topics_list=[10, 50, 200], device=device)
#model = topmost.models.TraCo(dataset.vocab_size, num_topics_list=[10, 50, 200])
model = model.to(device)

# create a trainer
trainer = topmost.trainers.HierarchicalTrainer(model, dataset)
# train the model
top_words, train_theta = trainer.train()

In [10]:
with open("/export/usuarios_ml4ds/lbartolome/Repos/my_repos/UserInLoopHTM/data/models/traco/trained_model_iter_1.pkl", "rb") as f:
    loaded_data = pickle.load(f)

results = loaded_data["hierarchy_quality_results"]
results

{'PCC': -0.1204091331866122,
 'PCD': 0.9680000000000001,
 'Sibling_TD': 0.9986666666666666,
 'PnCD': 0.9998518518518519}

In [ ]:
top_words

In [ ]:
########################### Evaluate ####################################
import json
import numpy as np
from topmost import evaluations

# get theta (doc-topic distributions)
train_theta, test_theta = trainer.export_theta()

# compute topic coherence
# refer to https://github.com/BobXWu/ECRTM

# compute topic diversity
TD = evaluations.multiaspect_topic_diversity(top_words)
print(f"TD: {TD}")

# evaluate clustering
#results = evaluations.hierarchical_clustering(test_theta, dataset.test_labels)
print(dict(results))

# evaluate classification
results = evaluations.hierarchical_classification(train_theta, test_theta, dataset.train_labels, dataset.test_labels)
print(dict(results))

In [ ]:
# evaluate quality of topic hierarchy
beta_list = trainer.get_beta()
phi_list = trainer.get_phi()
annoated_top_words = trainer.get_top_words(annotation=True)
reference_bow = np.concatenate((dataset.train_bow, dataset.test_bow), axis=0) # or reference_bow = train_bow
results, topic_hierarchy = evaluations.hierarchy_quality(dataset.vocab, reference_bow, annoated_top_words, beta_list, phi_list)

print(json.dumps(topic_hierarchy, indent=4))
print(results)